In [9]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# import face_utilities as face

# Define the base directory where the face dataset is located
base_directory = 'Dataset/VISA_Face/VISA_Face'
train_dir = "/Users/zhengweing/Desktop/Current/CSCI158/FaceAndIris_Recognition/Face_Output/Face_Output_Split_Train"
# Load training data
X_train = np.load(os.path.join(train_dir, 'X_train.npy'))
y_train = np.load(os.path.join(train_dir, 'y_train.npy'))

# Define the CNN architecture for facial recognition
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(400, 300, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(y_train)), activation='softmax')  # Output layer with softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
X_test = np.load(os.path.join("Face_Output/Face_Output_Split_Test", "X_test.npy"))
y_test = np.load(os.path.join("Face_Output/Face_Output_Split_Test", "y_test.npy"))
# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)


ValueError: Invalid dtype: str800